In [ ]:
# Train model
import sys
sys.path.append(r'D:\skin_disease_detection\backend\app')
# from models import pretrained_model

In [ ]:
# Import saved model
from keras.models import load_model
model = load_model(r'D:\skin_disease_detection\backend\app\static\trained_models\skin_disease_model.h5')
print(model)

In [ ]:
import sys
import pandas as pd
sys.path.append(r'D:\skin_disease_detection\backend\app')
from utils.preprocess import get_data_generators
from ai_models.ensemble_model import run_ensemble_model

# Prepare data generators
train_generator, validation_generator, test_generator, label_encoder, class_weights = get_data_generators()

# Run the ensemble model
run_ensemble_model(train_generator, validation_generator, test_generator, class_weights)

In [ ]:
# Checking the model prediction

import sys
import numpy as np
from tensorflow.keras.models import load_model
from joblib import load
from tensorflow.keras.preprocessing.image import load_img, img_to_array

sys.path.append(r'D:\skin_disease_detection\trained_models')

# Load base models
resnet_model = load_model(r"D:\skin_disease_detection\trained_models\ResNet50_model.h5")
print("ResNet50 model loaded successfully!")

efficientnet_model = load_model(r"D:\skin_disease_detection\trained_models\EfficientNetB0_model.h5")
print("EfficientNet model loaded successfully!")

densenet_model = load_model(r"D:\skin_disease_detection\trained_models\DenseNet121_model.h5")
print("DenseNet model loaded successfully!")

# # Load meta-model
meta_model = load(r"D:\skin_disease_detection\trained_models\meta_model.pkl")
print("Meta-model loaded successfully!")

# Load an image
image_path = r"D:\skin_disease_detection\backend\data\Ham10000\HAM10000_images_part_2\ISIC_0033475.jpg"
img = load_img(image_path, target_size=(224, 224))  # Resize as per model input size

print(img)

# Preprocess the image
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Convert single image to batch
img_array = img_array / 255.0 

# Get predictions from the base models
resnet_pred = resnet_model.predict(img_array)
efficientnet_pred = efficientnet_model.predict(img_array)
densenet_pred = densenet_model.predict(img_array)

print("ResNet50 Prediction:", resnet_pred)
print("EfficientNet Prediction:", efficientnet_pred)
print("DenseNet Prediction:", densenet_pred)

# Combine predictions for the meta-model
base_predictions = np.array([resnet_pred, efficientnet_pred, densenet_pred]).reshape(1, -1)

# Get the final prediction from the meta-model
meta_prediction = meta_model.predict(base_predictions)
print("Meta-Model Final Prediction:", meta_prediction)